In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install webob
!pip install finance-datareader
!pip install pytrends

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pytrends: filename=pytrends-4.8.0-py3-none-any.whl size=16126 sha256=dfe79f8406fbbded6d0bb797a816d297f5e8f955484e4e256b97d857785fc38a
  Stored in directory: /root/.cache/pip/wheels/07/6f/5c/8174f98dec1bfbc7d5da4092854afcbcff4b26c3d9b66b5183
Successfully built pytrends


In [31]:
import json, os, re, requests, sys, time, pickle, math
import pandas as pd
import numpy as np
import logging
import logging.handlers

from bs4 import BeautifulSoup as bs
from urllib.request import urlopen, Request
from datetime import date, datetime, timedelta
import datetime
from dateutil.relativedelta import relativedelta
# from pykrx import stock
from webob.compat import urlparse
import lxml

# 주가지표
import FinanceDataReader as fdr

# 구글 트랜드 지수 데이터
from pytrends.request import TrendReq


URL_NAVER_FINANCE = "https://finance.naver.com"

import plotly.graph_objects as go
import plotly.subplots as ms
import plotly.offline as pyo
# Set notebook made to work in offline
pyo.init_notebook_mode()

Output hidden; open in https://colab.research.google.com to view.

### 네이버 경제 지표

In [6]:
class GetFinanceData:
    def __init__(self):
        self.values_oil = []
        self.values_gold = []
        self.values_material1 = []
        self.values_material2 = []
        self.values_material3 = []
        self.values_exchage = []
        self.values_irr = []
        pass

    ### 원천 Data URL 수집 ####
    def get_finance_url(self):

        # 달러/원화 환율

        USDKRW_url = URL_NAVER_FINANCE + "/marketindex/exchangeDailyQuote.naver?marketindexCd=FX_USDKRW"


        # 국제시장 환율
        exchange_url = URL_NAVER_FINANCE + "/marketindex/worldExchangeList.naver?key=exchange&page=%s"

        self.values_exchage.append(['달러/대한민국 원', USDKRW_url])

        page = 1
        while page <= 5:
            r_url = exchange_url % page
            r = requests.get(r_url)
            soup = bs(r.text,'lxml')
            tbody = soup.find('tbody')
            # sym = tbody.find_all('td', {'class' : 'symbol'})
            tit = tbody.find_all('td', class_ = 'tit')
            tr = tbody.select('td.tit > a')
            symbols = tbody.find_all('td', class_ = 'symbol')

            page += 1

            for i, j, s  in zip(tit, tr, symbols):
                if any(s.text == x for x in ['USDCNY', 'GBPUSD','EURUSD','USDJPY','USDCAD','USDCHF','USDSEK']):
                    # print(s.text)
                    tit = i.text
                    n_url = URL_NAVER_FINANCE + j['href']
                    r1 = requests.get(n_url)
                    soup1 = bs(r1.text,'lxml')
                    try:
                        f_url = URL_NAVER_FINANCE + soup1.find('iframe', title = '일별시세')['src']
                    except:
                        f_url = re.sub('worldExchangeDetail',"worldDailyQuote",n_url)
                    self.values_exchage.append([tit, f_url])
                else: pass
            time.sleep(1)


        # 국내, 국제 유가 / 귀금속
        gold_url = URL_NAVER_FINANCE + '/marketindex/?tabSel=gold#tab_section'

        summary_list = ["유가 리스트", "귀금속 리스트"]

        for i, t_list in enumerate(summary_list):
            r = requests.get(gold_url)
            soup = bs(r.text,'lxml')
            tbody = soup.find('table', summary=t_list).find('tbody')
            tr = tbody.select('td.tit > a')

            for j in tr:
                tit = j.text
                n_url = URL_NAVER_FINANCE + j['href']
                r1 = requests.get(n_url)
                soup1 = bs(r1.text,'lxml')
                try:
                    if tit == '국내 금':
                        f_url = URL_NAVER_FINANCE + soup1.find('iframe', title = '일별시세')['src']+'?'
                    else: f_url = URL_NAVER_FINANCE + soup1.find('iframe', title = '일별시세')['src']
                except:
                    f_url = re.sub('worldOilDetail',"worldDailyQuote",n_url)
                if i == 0:
                    self.values_oil.append([tit,f_url])
                else: self.values_gold.append([tit,f_url])


        # 원자재
        materials_url = URL_NAVER_FINANCE + '/marketindex/?tabSel=materials#tab_section'

        summary_list = ["에너지 선물", "비철금속","농산물 선물"]

        for i, t_list in enumerate(summary_list):
            r = requests.get(materials_url)
            soup = bs(r.text,'lxml')
            tbody = soup.find('table', summary=t_list).find('tbody')
            tr = tbody.select('td.tit > a')
            # print(t_list)

            for j in tr:
                tit = j.text
                # print(i, tit, j)
                n_url = URL_NAVER_FINANCE + j['href']
                r1 = requests.get(n_url)
                soup1 = bs(r1.text,'lxml')
                try:
                    f_url = URL_NAVER_FINANCE + soup1.find('iframe', title = '일별시세')['src']
                    f_url = re.sub('&page=1','',f_url)
                except:
                    f_url = re.sub('materialDetail','worldDailyQuote',n_url)
                    f_url = re.sub('marketindexCd','fdtc=2&marketindexCd',f_url)

                if i == 0:
                    self.values_material1.append([tit,f_url])
                elif i == 1:
                    self.values_material2.append([tit,f_url])
                else: self.values_material3.append([tit,f_url])


        # 국내 시장 금리
        IRR_list = ['CD91','CALL','GOVT03Y','CORP03Y']
        IRR_COFIX = [['COFIX잔액',URL_NAVER_FINANCE + '/marketindex/interestDailyQuote.naver?marketindexCd=IRR_COFIXBAL'],
                     ['COFIX신규취급액',URL_NAVER_FINANCE + '/marketindex/interestDailyQuote.naver?marketindexCd=IRR_COFIXNEW']]

        for i in IRR_list:   
            IRR_url = i+'_url'
            IRR_url = URL_NAVER_FINANCE + '/marketindex/interestDetail.naver?marketindexCd=IRR_'+ i
            r = requests.get(IRR_url)
            soup = bs(r.text,'lxml')
            f_url = URL_NAVER_FINANCE + soup.find('iframe', title = '일별시세')['src']
            tit = soup.find('div', id="container").find('h2')
            values_IRR = 'values_'+i
            values_IRR = [tit.text, f_url]
            self.values_irr.append(values_IRR)


        # 국내 시장 금리(values_irr)에 COFIX지표 추가
        for j in range(len(IRR_COFIX)):
            f_url = IRR_COFIX[j]
            self.values_irr.append(f_url)


        # 추출 항목 url_dict 에등록
        url_dict = {'유가': dict(self.values_oil),
                    '귀금속': dict(self.values_gold),
                    '에너지': dict(self.values_material1),
                    '비철금속': dict(self.values_material2),          	
                    '농산물': dict(self.values_material3),			
                    '국제시장환율': dict(self.values_exchage),
                    '국내시장금리': dict(self.values_irr)
                   }
        return url_dict


    #### 수집한 url로 부터 자료 추출 ####
    def get_finance_data(self, input_page=3):

        url_dict = self.get_finance_url()    
        url_list = list(url_dict.keys())

        df = pd.DataFrame()

        for dict_key in list(url_dict.keys()):

            for key in url_dict[dict_key].keys():

                values = []

                page = 1
                page_text_len = 100

                # print(key+ '자료 수집 시작 ####')

                while page < input_page: # page_text_len > 10:
                    r_url = url_dict[dict_key][key] + '&page=%s' % page
                    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.0.0 Safari/537.36'}
                    # print('r_url = ',r_url)
                    r = requests.get(r_url, headers = headers)
                    soup = bs(r.text,'lxml')


        #             if dict_key == '주식':

        #                 tr1 = soup.find_all('span', class_= 'tah p10 gray03')
        #                 tr2 = soup.find_all('td', class_= 'num')

        #                 for i, t in enumerate(tr1):
        #                     x = t.text.strip().replace('+','').replace('\t','').split('\n')
        #                     x = ' '.join(x).split()

        #                     i = i * 6
        #                     y_list = []
        #                     for j in range(i+0, i+6):
        #                         y = tr2[j].text.strip().replace('+','').replace('\t','').split('\n')
        #                         y = ' '.join(y).split()
        #                         y_list.extend(y)

        #                     x.extend(y_list)
        #                     # print(x)
        #                     values.append(x)

        #                 page += 1

                    # else:

                    tbody = soup.find('tbody')
                    tr = tbody.find_all('tr')

                    page_text_len = len(tbody.text.strip())
                    # print(page_text_len)

                    for t in tr:
                        x = t.text.strip().replace('+','').replace('\t','').split('\n')
                        x = ' '.join(x).split()
                        values.append(x)

                    page += 1

                # 전일대비값 : text->flaot 변환 및 등락율의 부호에 따라 부호 수정
                # 종가 : text->flaot 변환

                for i in values:
                    i[1] = float(i[1].replace(',',''))
                    i[2] = float(i[2].replace(',',''))
                    if i[3][0] == "-":
                        i[2] = i[2] * (-1)
                    else: pass


                # print(values)

                date = []
                key_values = []
                for i in range(len(values)):
                    x = values[i][0] # 일자
                    y = values[i][1] # 종가
                    date.append(x)
                    key_values.append(y)

                df_temp = pd.DataFrame(key_values, index=date, columns=[key])
                df = pd.merge(df, df_temp, how='outer', left_index=True, right_index=True)

                print(f"{key} + 자료 수집 완료 #### : {len(df_temp)}")
                time.sleep(3)
                
        df = df.sort_index(ascending=False)      
        df = df.reset_index().rename(columns = {'index':'date'})
        df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

        return df

In [7]:
gfd = GetFinanceData()
url_dict = gfd.get_finance_url()
print(url_dict.keys())
df = gfd.get_finance_data(300)

dict_keys(['유가', '귀금속', '에너지', '비철금속', '농산물', '국제시장환율', '국내시장금리'])
휘발유 + 자료 수집 완료 #### : 2093
고급휘발유 + 자료 수집 완료 #### : 2093
경유 + 자료 수집 완료 #### : 2093
두바이유 + 자료 수집 완료 #### : 2093
브렌트유 + 자료 수집 완료 #### : 2093
WTI(서부텍사스유)  + 자료 수집 완료 #### : 2093
국내 금 + 자료 수집 완료 #### : 2990
국제 금 + 자료 수집 완료 #### : 2093
백금 + 자료 수집 완료 #### : 2093
은 + 자료 수집 완료 #### : 2093
팔라듐 + 자료 수집 완료 #### : 2093
가스오일 + 자료 수집 완료 #### : 2093
난방유 + 자료 수집 완료 #### : 2093
천연가스 + 자료 수집 완료 #### : 2093
구리 + 자료 수집 완료 #### : 2093
납 + 자료 수집 완료 #### : 2093
아연 + 자료 수집 완료 #### : 2093
니켈 + 자료 수집 완료 #### : 2093
알루미늄합금 + 자료 수집 완료 #### : 2093
주석 + 자료 수집 완료 #### : 2093
옥수수 + 자료 수집 완료 #### : 2093
설탕 + 자료 수집 완료 #### : 2093
대두 + 자료 수집 완료 #### : 2093
대두박 + 자료 수집 완료 #### : 2093
대두유 + 자료 수집 완료 #### : 2093
면화 + 자료 수집 완료 #### : 2093
소맥 + 자료 수집 완료 #### : 2093
쌀 + 자료 수집 완료 #### : 2093
오렌지주스 + 자료 수집 완료 #### : 2093
커피 + 자료 수집 완료 #### : 2093
코코아 + 자료 수집 완료 #### : 2093
달러/대한민국 원 + 자료 수집 완료 #### : 2990
영국 파운드/달러 + 자료 수집 완료 #### : 2093
유로/달러 + 자료 수집 완료 #### : 2

In [24]:
# 저장
df_finance = df[df['date'].dt.strftime('%Y%m%d').str[0:6] >= '201704']
df_finance.to_csv ("/content/drive/MyDrive/Colab Notebooks/실무인증/df_finance_20220528.csv", index = None)

### <br>주가지표

In [26]:
# 암호화폐 가격에 영향을 미칠 GPU 관련 업체(NVIDIA/AMD)와 비디오메모리 생산업체(삼성전자/하이닉스) 의 주가지표 수집

# KRX : KRX 종목 전체
# KOSPI : KOSPI 종목
# KOSDAQ : KOSDAQ 종목
# KONEX : KONEX 종목
# NASDAQ : 나스닥 종목
# NYSE : 뉴욕증권거래소 종목
# SP500 : S&P500 종목

df_krx = fdr.StockListing('KRX')
df_NASDAQ = fdr.StockListing('NASDAQ')

# df_krx.loc[df_krx['Name'] == '삼성전자']['Symbol'] '005930'
# df_krx.loc[df_krx['Name'] == 'SK하이닉스']['Symbol'] '000660'
# df_NASDAQ.loc[df_NASDAQ['Name'] == 'NVIDIA Corp']['Symbol'] 'NVDA'
# df_NASDAQ.loc[df_NASDAQ['Name'] == 'Advanced Micro Devices Inc']['Symbol'] 'AMD'


Samsung_df = fdr.DataReader('005930','20170501','20220528')
Hynix_df = fdr.DataReader('000660','20170501','20220528')
NVDA_df = fdr.DataReader('NVDA','20170501','20220528')
AMD_df = fdr.DataReader('AMD','20170501','20220528')

ss_df = Samsung_df[['Close','Volume']]
ss_df.columns = ['samsung_trade_price','samsung_trade_volume']

hn_df = Hynix_df[['Close','Volume']]
hn_df.columns = ['hynix_trade_price','hynix_trade_volume']

nv_df = NVDA_df[['Close','Volume']]
nv_df.columns = ['nvdia_trade_price','nvdia_trade_volume']

amd_df = AMD_df[['Close','Volume']]
amd_df.columns = ['amd_trade_price','amd_trade_volume']

stk_df = pd.concat([ss_df, hn_df, nv_df, amd_df], axis=1)
df_stock = stk_df.reset_index().rename(columns = {'Date':'date'})
df_stock.head()

100%|██████████| 4623/4623 [00:16<00:00, 283.82it/s]


,date,samsung_trade_price,samsung_trade_volume,hynix_trade_price,hynix_trade_volume,nvdia_trade_price,nvdia_trade_volume,amd_trade_price,amd_trade_volume
0,2017-05-01,NaN,NaN,NaN,NaN,26.66,30770000.0,13.62,68040000.0
1,2017-05-02,44900.0,281366.0,55400.0,4347745.0,25.87,63000000.0,10.32,268340000.0
2,2017-05-03,NaN,NaN,NaN,NaN,26.06,33690000.0,10.39,140900000.0
3,2017-05-04,45520.0,273802.0,55900.0,2780438.0,25.96,20980000.0,10.10,77030000.0
4,2017-05-05,NaN,NaN,NaN,NaN,25.96,22840000.0,10.19,51760000.0


In [29]:
# 저장
df_stock.to_csv ("/content/drive/MyDrive/Colab Notebooks/실무인증/df_stock_20220528.csv", index = None)

### 암호화폐 공포탐욕지수

In [32]:
# 암호화폐 공포탐욕지수

url = ' https://api.alternative.me/fng/?limit=0'
data = requests.get(url).json()
value = []
time = []
for i in data['data']:
    value.append(i['value'])
    time.append(datetime.datetime.fromtimestamp(int(i['timestamp'])).strftime('%Y-%m-%d'))
value = value[::-1]
time = pd.to_datetime(time[::-1])
fng = pd.Series(value,time,name='fng')

df_fng = pd.DataFrame(fng).reset_index().rename(columns={'index':'date'})
df_fng['fng'] = df_fng['fng'].astype(int) 
df_fng.head()

,date,fng
0,2018-02-01,30
1,2018-02-02,15
2,2018-02-03,40
3,2018-02-04,24
4,2018-02-05,11


In [33]:
# 저장
df_fng.to_csv ("/content/drive/MyDrive/Colab Notebooks/실무인증/df_fng_20220528.csv", index = None)

### 구글 트랜드 지수

In [34]:
# 구글 트랜드 지수 데이터

pytrends = TrendReq(hl='', tz=540)
keywords = ['이더리움']
pytrends.build_payload(keywords, cat=0, timeframe='today 5-y', geo='', gprop='')
df_trend = pytrends.interest_over_time()
df_trend = df_trend.drop(columns='isPartial')
df_trend.columns = ['이더리움_구글트랜드']
df_trend = pd.DataFrame(df_trend).reset_index()
df_trend.tail()

,date,이더리움_구글트랜드
255,2022-04-24,22
256,2022-05-01,23
257,2022-05-08,28
258,2022-05-15,26
259,2022-05-22,22


In [35]:
# 저장
df_trend.to_csv ("/content/drive/MyDrive/Colab Notebooks/실무인증/df_trend_20220528.csv", index = None)

## Merge DataFrame
#### (네이버 파이낸스 + 주가 + 공포탐욕지수 + 구글 검색 트랜드 지수)<br>

In [37]:
df_1 = pd.merge(df, df_stock,  how='outer', on='date')
df_2 = pd.merge(df_1, df_fng,  how='outer', on='date')
df_new = pd.merge(df_2, df_trend,  how='outer', on='date')
# 저장
df_new.to_csv ("/content/drive/MyDrive/Colab Notebooks/실무인증/df_new_20220528.csv", index = None)
df_new

,date,휘발유,고급휘발유,경유,두바이유,브렌트유,WTI(서부텍사스유),국내 금,국제 금,백금,...,samsung_trade_price,samsung_trade_volume,hynix_trade_price,hynix_trade_volume,nvdia_trade_price,nvdia_trade_volume,amd_trade_price,amd_trade_volume,fng,이더리움_구글트랜드
0,2022-05-29,2008.84,2226.22,2006.10,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN
1,2022-05-28,2007.85,2225.83,2005.71,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN
2,2022-05-27,2004.87,2223.46,2005.13,110.86,115.56,115.07,74845.11,1851.3,945.5,...,66500.0,11346700.0,106000.0,3666691.0,188.11,73100000.0,102.26,104010000.0,12.0,NaN
3,2022-05-26,2001.82,2219.47,2003.86,109.78,114.17,114.09,75176.73,1847.6,939.9,...,65900.0,15970890.0,103000.0,8523001.0,178.51,99080000.0,98.75,128520000.0,12.0,NaN
4,2022-05-25,1998.96,2217.03,2002.77,106.92,111.12,110.33,75479.26,1846.3,931.8,...,66400.0,15150490.0,108000.0,4151560.0,169.75,72170000.0,92.65,109880000.0,11.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,2017-04-05,1487.95,1817.98,1278.20,52.50,54.36,51.15,45464.98,1245.4,956.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1881,2017-04-04,1488.77,1818.64,1279.16,52.19,54.17,51.03,45478.72,1255.0,961.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1882,2017-04-03,1489.61,1818.29,1280.12,51.35,53.12,50.24,44836.88,1250.8,954.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1883,2017-04-02,1490.45,1819.37,1280.63,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1885 entries, 0 to 1884
Data columns (total 56 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  1885 non-null   datetime64[ns]
 1   휘발유                   1885 non-null   float64       
 2   고급휘발유                 1885 non-null   float64       
 3   경유                    1885 non-null   float64       
 4   두바이유                  1301 non-null   float64       
 5   브렌트유                  1327 non-null   float64       
 6   WTI(서부텍사스유)           1300 non-null   float64       
 7   국내 금                  1271 non-null   float64       
 8   국제 금                  1299 non-null   float64       
 9   백금                    1299 non-null   float64       
 10  은                     1299 non-null   float64       
 11  팔라듐                   1299 non-null   float64       
 12  가스오일                  1326 non-null   float64       
 13  난방유               

In [ ]:
# df_1.sort_index(ascending=False).reset_index(drop=False).rename(columns = {'index':'일자'})

# %time df_1.to_pickle('oildata.pickle')

In [40]:
df_topic_news = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/실무인증/Data/df_topic_news_20220528.csv')


In [47]:
df_fn_tp = pd.merge(df_topic_news, df_new,  how='outer', on='date')
df_fn_tp.head()

ValueError: ignored

In [44]:
df_topic_news = df_topic_news[['date','Dominant_Topic','Topic_Perc_Contrib']]